In [1]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.4-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.4-py3-none-any.whl (240 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp310-cp310-linux_x86_64.whl size=4246490 sha256=eb1a40c7c35e0342f8595cf0ee0dea7ca0356906bcd42a576a8d6268273e0181
  Stored in directory: /root/.cache/pip/wheels/0d/a2/00/81db54d3e6a8199b829d58e02cec2ddb20ce3e59fad8d3c92a
Successfully built fasttext


In [2]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('products.csv')
df.head()

,Unnamed: 0,product_name,brand_name,rating,rating_count,marked_price,discounted_price,sizes,product_link,img_link,product_tag,brand_tag,discount_amount,discount_percent,tags,seller_address,meta_description,meta_keyword
0,0,Slim Fit Casual Shirt,Greenfibre,0.0,0,1699,1699,"38,39,40,42,44",shirts/greenfibre/greenfibre-men-yellow-slim-f...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",shirts,greenfibre,0,0,"clothing,men-clothing,shirts,greenfibre-shirts",Gujarat - 380006,Buy Greenfibre Men Yellow Slim Fit Checked Cas...,Greenfibre Men Yellow Slim Fit Checked Casual ...
1,1,Men Printed T-shirt,FREESOUL,0.0,0,799,799,"S,M,L,XL,XXL",tshirts/freesoul/freesoul-men-white-printed-co...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",tshirts,freesoul,0,0,"clothing,men-clothing,tshirts,freesoul-tshirts",Karnataka - 560058,Buy FREESOUL Men White Printed Cotton T Shirt ...,"FREESOUL Men White Printed Cotton T Shirt, FR..."
2,2,Leather Two Fold Wallet,Da Milano,0.0,0,4499,4499,Onesize,wallets/da-milano/da-milano-women-blue-texture...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",wallets,da-milano,0,0,"accessories,women-accessories,wallets,da-milan...",Delhi - 110037,Buy Da Milano Women Blue Textured Leather Two ...,Da Milano Women Blue Textured Leather Two Fold...
3,3,Pure Georgette Bandhani Saree,Mitera,3.6,5,6727,1681,Onesize,sarees/mitera/mitera-women-sea-green--white-em...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",sarees,mitera,5046,75,"clothing,women-clothing,sarees,mitera-sarees",Gujarat - 395010,Buy Mitera Women Sea Green & White Embroidered...,Mitera Women Sea Green & White Embroidered Pur...
4,4,Men Geometric Printed Kurta,Indo Era,0.0,0,2599,1429,"S,M,L,XL,XXL",kurtas/indo-era/indo-era-men-lavender-geometri...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",kurtas,indo-era,1170,45,"clothing,men-clothing,kurtas,indo-era-kurtas",Gujarat - 395010,Buy Indo Era Men Lavender Geometric Printed Ku...,"Indo Era Men Lavender Geometric Printed Kurta,..."


In [12]:
df['features'] = df['product_name'] + ' ' +  df['product_tag'] + ' ' + df['brand_tag']
df['tags'] = ','+df['tags']
df['labels'] = [ ' __label__'.join(tag.split(',')).strip().lower() for tag in df['tags']]
df.head(2)

,Unnamed: 0,product_name,brand_name,rating,rating_count,marked_price,discounted_price,sizes,product_link,img_link,product_tag,brand_tag,discount_amount,discount_percent,tags,seller_address,meta_description,meta_keyword,features,labels
0,0,Slim Fit Casual Shirt,Greenfibre,0.0,0,1699,1699,"38,39,40,42,44",shirts/greenfibre/greenfibre-men-yellow-slim-f...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",shirts,greenfibre,0,0,",,,clothing,men-clothing,shirts,greenfibre-shirts",Gujarat - 380006,Buy Greenfibre Men Yellow Slim Fit Checked Cas...,Greenfibre Men Yellow Slim Fit Checked Casual ...,Slim Fit Casual Shirt shirts greenfibre,__label__ __label__ __label__clothing __label_...
1,1,Men Printed T-shirt,FREESOUL,0.0,0,799,799,"S,M,L,XL,XXL",tshirts/freesoul/freesoul-men-white-printed-co...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",tshirts,freesoul,0,0,",,,clothing,men-clothing,tshirts,freesoul-tshirts",Karnataka - 560058,Buy FREESOUL Men White Printed Cotton T Shirt ...,"FREESOUL Men White Printed Cotton T Shirt, FR...",Men Printed T-shirt tshirts freesoul,__label__ __label__ __label__clothing __label_...


In [13]:
data = [ i[-1]+' '+i[-2] for i in df.values]

fd = open('data.txt','w')
for i in data:
  fd.write(i+'\n')
fd.close()

In [15]:
import fasttext
model = fasttext.train_supervised(input='data.txt')

In [25]:
for i in range(10):
   txt = df['features'][np.random.randint(len(df['features']))]
   print(model.predict(txt.lower(),k=5),' ',txt)

(('__label__', '__label__clothing', '__label__men-clothing', '__label__women-clothing', '__label__shirts'), array([0.5740537 , 0.1128784 , 0.0282817 , 0.02008706, 0.00829935]))   Slim Fit Casual Shirt shirts highlander
(('__label__', '__label__clothing', '__label__women-clothing', '__label__kurtas', '__label__men-clothing'), array([0.29824466, 0.0698371 , 0.03223717, 0.01024364, 0.00900631]))   Women Floral Printed Kurta kurtas soch
(('__label__women-footwear', '__label__footwear', '__label__', '__label__flats', '__label__heels'), array([0.30563846, 0.28520295, 0.13209376, 0.08164282, 0.02975981]))   Women Beaded Open Toe Flats flats catwalk
(('__label__', '__label__clothing', '__label__women-clothing', '__label__personal-care', '__label__unisex-home-furnishing'), array([0.76178366, 0.08831205, 0.07469904, 0.00502582, 0.0039741 ]))   Solid Playsuit jumpsuit sassafras
(('__label__', '__label__clothing', '__label__women-clothing', '__label__kurta-sets', '__label__unisex-home-furnishing')